In [73]:
import pandas as pd
import numpy as np
import string


data_frame = pd.read_csv("SMSSpamCollection", sep="\t" ,header = None, names=["label","message"])
display(data_frame)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [62]:
total = data_frame.shape[0]
ham = data_frame.loc[data_frame['label'] == 'ham']
ham_counts = ham.shape[0]
spam = data_frame.loc[data_frame['label'] == 'spam']
spam_counts = spam.shape[0]

In [63]:
print('total documents: ', total, 'lines')
print('ham: ', ham_counts, 'lines')
print('spam: ', spam_counts, 'lines')

total documents:  5572 lines
ham:  4825 lines
spam:  747 lines


In [74]:
print(data_frame['label'].value_counts())

label
ham     4825
spam     747
Name: count, dtype: int64


In [65]:
i = 1
for message in data_frame['message']:
    words = message.split()
    total_words = len(words)
    print('message ',i,': ',total_words)
    i+=1

message  1 :  20
message  2 :  6
message  3 :  28
message  4 :  11
message  5 :  13
message  6 :  32
message  7 :  16
message  8 :  26
message  9 :  26
message  10 :  29
message  11 :  21
message  12 :  26
message  13 :  26
message  14 :  37
message  15 :  8
message  16 :  19
message  17 :  4
message  18 :  19
message  19 :  13
message  20 :  24
message  21 :  8
message  22 :  11
message  23 :  11
message  24 :  20
message  25 :  11
message  26 :  28
message  27 :  5
message  28 :  32
message  29 :  15
message  30 :  12
message  31 :  26
message  32 :  41
message  33 :  6
message  34 :  17
message  35 :  29
message  36 :  28
message  37 :  9
message  38 :  8
message  39 :  4
message  40 :  30
message  41 :  17
message  42 :  34
message  43 :  33
message  44 :  4
message  45 :  13
message  46 :  3
message  47 :  8
message  48 :  5
message  49 :  14
message  50 :  30
message  51 :  12
message  52 :  23
message  53 :  16
message  54 :  55
message  55 :  22
message  56 :  13
message  57 : 

In [82]:
def preprocess(message):
    message = message.lower()
    message = message.translate(str.maketrans('','', string.punctuation))
    return message.split()

In [83]:
documents = []
for message in data_frame['message']:
    words = preprocess(message)
    documents.append(words)

vocab = set()
for doc in documents:
    vocab.update(doc)
vocab = sorted(list(vocab))

print(vocab)

vocab_index = {}
for i in range(len(vocab)):
    vocab_index[vocab[i]] = i

def td(doc):
    td_vector = np.zeros(len(vocab))
    for word in doc:
        td_vector[vocab_index[word]] +=1
    td_vector = td_vector/len(doc)
    return td_vector

def idf(documents):
    idf = np.zeros(len(vocab))
    for doc in documents:
        seen = set()
        for word in doc:
            if word not in seen:
                idf[vocab_index[word]] += 1
                seen.add(word)
    idf = np.log(len(documents)/(idf))
    return idf

idf_vector = idf(documents)
td_table = np.array([td(doc) for doc in documents])

td_idf = td_table * idf_vector
td_idf_df = pd.DataFrame(td_idf, columns = vocab)
display(td_idf_df)



['0', '008704050406', '0089my', '0121', '01223585236', '01223585334', '0125698789', '02', '020603', '0207', '02070836089', '02072069400', '02073162414', '02085076972', '020903', '021', '050703', '0578', '06', '060505', '061104', '07008009200', '07046744435', '07090201529', '07090298926', '07099833605', '071104', '07123456789', '0721072', '07732584351', '07734396839', '07742676969', '07753741225', '0776xxxxxxx', '07786200117', '077xxx', '078', '07801543489', '07808', '07808247860', '07808726822', '07815296484', '07821230901', '0784987', '0789xxxxxxx', '0794674629107880867867', '0796xxxxxx', '07973788240', '07xxxxxxxxx', '0800', '08000407165', '08000776320', '08000839402', '08000930705', '08000938767', '08001950382', '08002888812', '08002986030', '08002986906', '08002988890', '08006344447', '0808', '08081263000', '08081560665', '0825', '0844', '08448350055', '08448714184', '0845', '08450542832', '08452810071', '08452810073', '08452810075over18s', '0870', '08700621170150p', '08701213186',

C:\Users\maria\AppData\Local\Temp\ipykernel_2528\1778066579.py:21: RuntimeWarning: invalid value encountered in divide
  td_vector = td_vector/len(doc)


,0,008704050406,0089my,0121,01223585236,01223585334,0125698789,02,020603,0207,...,£s,»10,é,ü,üll,–,“,“harry,…,…thanks
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
5568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.465959,0.0,0.0,0.0,0.0,0.0,0.0
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
